In [ ]:
import findspark
findspark.init()

import folium

from folium.plugins import MarkerCluster
from pyspark import sql, SparkConf, SparkContext
from ipywidgets import interact

In [ ]:
conf = SparkConf().setAppName("Read_CSV")
sc = SparkContext(conf=conf)
sqlContext = sql.SQLContext(sc)

dfCities = sqlContext.read.csv("Steden.csv", header=True)
dfPollution = sqlContext.read.csv("Luchtvervuiling.csv", header=True)

coordinates = (20.593684, 78.96288)
pMap = folium.Map(location=coordinates, zoom_start=4)

In [ ]:
TYPES_P = ["so2", "no2", "rspm", "spm"]

STRONG_BLUE = "#3186cc"
RED = "#ff0000"
GREEN = "#7CFC00"
LIGHT_ORANGE = "#ffd27f"

In [ ]:
dfMapCities = dfCities.select("lat", "lng", "city")
dfMapPollution = dfPollution.select("location", "so2", "no2", "rspm", "spm")
colorPoint = STRONG_BLUE

@interact(types=TYPES_P)
def get_pollution(types):
    index = 0
    joinDF = dfMapPollution.join(dfMapCities, dfMapPollution.location == dfMapCities.city, how="right")
    if types == "so2":
        joinDF = joinDF.filter("so2 != ''").filter("so2 != 'NA'").filter("lat != ''")\
        .filter("lng != ''").dropDuplicates(["city"]).collect()
    elif types == "no2":
        joinDF = joinDF.filter("no2 != ''").filter("no2 != 'NA'").filter("lat != ''")\
        .filter("lng != ''").dropDuplicates(["city"]).collect()
    elif types == "rspm":
        joinDF = joinDF.filter("rspm != ''").filter("rspm != 'NA'").filter("lat != ''")\
        .filter("lng != ''").dropDuplicates(["city"]).collect()
    elif types == "spm":
        joinDF = joinDF.filter("spm != ''").filter("spm != 'NA'").filter("lat != ''")\
        .filter("lng != ''").dropDuplicates(["city"]).collect()
        
    for line in joinDF:
        if types == "so2":
            pollution = joinDF[index][1]
            if float(pollution) > 10:
                colorPoint = RED
            elif float(pollution) < 5:
                colorPoint = GREEN
            else:
                colorPoint = LIGHT_ORANGE
        elif types == "no2":
            pollution = joinDF[index][2]
            if float(pollution) > 18:
                colorPoint = RED
            elif float(pollution) < 10:
                colorPoint = GREEN
            else:
                colorPoint = LIGHT_ORANGE
        elif types == "rspm":
            pollution = joinDF[index][3]
            if float(pollution) > 200:
                colorPoint = RED
            elif float(pollution) < 80:
                colorPoint = GREEN
            else:
                colorPoint = LIGHT_ORANGE
        elif types == "spm":
            pollution = joinDF[index][4]
            if float(pollution) > 300:
                colorPoint = RED
            elif float(pollution) < 200:
                colorPoint = GREEN
            else:
                colorPoint = LIGHT_ORANGE

        folium.CircleMarker(
            location = [joinDF[index][5], joinDF[index][6]],
            radius = 4,
            popup = pollution,
            color = colorPoint,
            fill = True,
            fill_color = STRONG_BLUE
        ).add_to(pMap)
        index = index + 1
    return pMap